In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LambdaCallback
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
import tqdm
import random
from bayes_opt import BayesianOptimization

Using TensorFlow backend.


Load and clean.

In [2]:
data = pd.read_csv('final_data.csv')
# Convert labels
data.loc[data.CONT == "NO","CONT"] = 0
data.loc[data.CONT != 0,"CONT"] = 1
# Subset columns
cols = ["BUN","CA","CL","CO2","CRET","GLU","K","NA.","CONT"]
data = data[cols]
# Banish negativity :)
data.loc[data.CA < 0,"CA"] = 0
# Convert to int.  Could save space with binary, but meh...
data.CONT = data.CONT.astype("int64")
# Drop NAs
data = data.dropna()
# To numpy, Batman!
data = data.as_matrix()
# Normalize features
num_features = len(cols)-1
means = np.zeros(num_features)
stds = np.zeros(num_features)
for x in range(data.shape[1]-1):
    means[x] = data[:,x].mean()
    stds[x] = data[:,x].std()
    data[:,x] = (data[:,x]-means[x])/stds[x]
# Split
test_samples, val_samples = [int(.2*len(data))]*2
np.random.shuffle(data)
test = data[:test_samples,:]
val = data[test_samples:test_samples+val_samples,:]
train = data[test_samples+val_samples:,:]

In [3]:
def buildNet(lr=1e-3, dropoutA=.2, dropoutB=.2, dropoutC=.2):
    inputLayer = Input(shape = (8,))
    D = Dropout(dropoutA)(inputLayer)
    D = Dense(64)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutB)(D)
    D = Dense(32)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutC)(D)
    pred = Dense(1,activation = 'sigmoid')(D)
    model = Model(input = inputLayer, output=pred)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
    return model

def trainAndPredict(model, nb_epoch=100, batch_size=len(train)):
    #plotterT = LambdaCallback(on_epoch_end=lambda epoch, logs: plt.plot(epoch, logs['loss']))
    #plotterV = LambdaCallback(on_epoch_end=lambda epoch, logs: plt.plot(np.arange(epoch), logs['val_loss']))
    stopper = EarlyStopping(patience = 3, verbose = 1)
    model.fit(train[:,:8],train[:,-1],validation_data=(val[:,:8],val[:,-1]),
              nb_epoch=int(round(nb_epoch)), batch_size=int(round(batch_size)),callbacks=[stopper])
    return model.predict(val[:,:8])

def evalAUC(y_pred,y_val=val[:,-1]):
    return roc_auc_score(y_val,y_pred)

def measurable(lr=1e-3, dropoutA=.2, dropoutB=.2, dropoutC=.2, nb_epoch=100, batch_size=len(train)):
    model = buildNet(lr,dropoutA,dropoutB,dropoutC)
    y_pred = trainAndPredict(model, nb_epoch,batch_size)
    return evalAUC(y_pred)

In [ ]:
params = {'lr':(1e-5,.01), 'dropoutA':(.01,.6),'dropoutB':(.01,.6),'dropoutC':(.01,.6)}
BO = BayesianOptimization(measurable, params)
BO.maximize(init_points=5,n_iter=50)

In [25]:
BO.res['max']

{'max_params': {'dropoutA': 0.083730742972679156,
  'dropoutB': 0.37669203808894514,
  'dropoutC': 0.26408076037832368,
  'lr': 0.019685168087984706},
 'max_val': 0.99911993724769932}

In [153]:
model = buildNet(lr = 0.019685168087984706, dropoutA=0.083730742972679156, 
                 dropoutB=0.37669203808894514,dropoutC=0.26408076037832368)
y_pred = trainAndPredict(model)

Train on 8367 samples, validate on 2789 samples
Epoch 1/100
8367/8367 [==============================] - 0s - loss: 0.6157 - val_loss: 0.3245
Epoch 2/100
8367/8367 [==============================] - 0s - loss: 0.3484 - val_loss: 0.2088
Epoch 3/100
8367/8367 [==============================] - 0s - loss: 0.2350 - val_loss: 0.1613
Epoch 4/100
8367/8367 [==============================] - 0s - loss: 0.1914 - val_loss: 0.1383
Epoch 5/100
8367/8367 [==============================] - 0s - loss: 0.1624 - val_loss: 0.1198
Epoch 6/100
8367/8367 [==============================] - 0s - loss: 0.1533 - val_loss: 0.0993
Epoch 7/100
8367/8367 [==============================] - 0s - loss: 0.1260 - val_loss: 0.0824
Epoch 8/100
8367/8367 [==============================] - 0s - loss: 0.1031 - val_loss: 0.0703
Epoch 9/100
8367/8367 [==============================] - 0s - loss: 0.0942 - val_loss: 0.0634
Epoch 10/100
8367/8367 [==============================] - 0s - loss: 0.0820 - val_loss: 0.0598
Epoch 11/10

In [154]:
evalAUC(y_pred)

0.97354072203409281

In [88]:
def buildDeep(lr=.01, dropoutA=.2, dropoutB=.2, dropoutC=.2, dropoutD=.2,dropoutE=.2):
    inputLayer = Input(shape = (8,))
    D = Dropout(dropoutA)(inputLayer)
    D = Dense(64)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutB)(D)
    D = Dense(32)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutC)(D)
    D = Dense(16)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutD)(D)
    D = Dense(8)(D)
    D = Activation('relu')(D)
    D = Dropout(dropoutE)(D)
    pred = Dense(1,activation = 'sigmoid')(D)
    model = Model(input = inputLayer, output=pred)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
    return model

def measurableDeep(lr=1e-3, dropoutA=.2, dropoutB=.2, dropoutC=.2, dropoutD=.2, 
                   dropoutE=.2,nb_epoch=100, batch_size=len(train)):
    model = buildDeep(lr,dropoutA,dropoutB,dropoutC,dropoutD,dropoutE)
    y_pred = trainAndPredict(model, nb_epoch,batch_size)
    return evalAUC(y_pred)

In [ ]:
params1 = {'lr':(1e-3,.1), 'dropoutA':(.01,.6),'dropoutB':(.01,.6),'dropoutC':(.01,.6),
            'dropoutD':(.01,.6),'dropoutE':(.01,.6)}
BO1 = BayesianOptimization(measurableDeep,params1)
BO1.maximize(init_points=5,n_iter=50)

In [47]:
BO1.res['max']

{'max_params': {'dropoutA': 0.30412844220674939,
  'dropoutB': 0.38286540709231481,
  'dropoutC': 0.19078598982799802,
  'dropoutD': 0.21326732936366835,
  'dropoutE': 0.092861776007691854,
  'lr': 0.074875486052876028},
 'max_val': 0.99944517783007136}

In [141]:
deep = buildDeep(lr=0.074875486052876028,dropoutA=0.30412844220674939,
                 dropoutB=0.38286540709231481,dropoutC=0.19078598982799802,
                 dropoutD=0.21326732936366835,dropoutE=0.092861776007691854)
y_deep = trainAndPredict(deep)

Train on 8367 samples, validate on 2789 samples
Epoch 1/100
8367/8367 [==============================] - 0s - loss: 0.7302 - val_loss: 0.2458
Epoch 2/100
8367/8367 [==============================] - 0s - loss: 0.2665 - val_loss: 0.1711
Epoch 3/100
8367/8367 [==============================] - 0s - loss: 0.2085 - val_loss: 0.1583
Epoch 4/100
8367/8367 [==============================] - 0s - loss: 0.2100 - val_loss: 0.1373
Epoch 5/100
8367/8367 [==============================] - 0s - loss: 0.1836 - val_loss: 0.1161
Epoch 6/100
8367/8367 [==============================] - 0s - loss: 0.1642 - val_loss: 0.0956
Epoch 7/100
8367/8367 [==============================] - 0s - loss: 0.1464 - val_loss: 0.0850
Epoch 8/100
8367/8367 [==============================] - 0s - loss: 0.1255 - val_loss: 0.0783
Epoch 9/100
8367/8367 [==============================] - 0s - loss: 0.0994 - val_loss: 0.0720
Epoch 10/100
8367/8367 [==============================] - 0s - loss: 0.1018 - val_loss: 0.0679
Epoch 11/10

In [143]:
evalAUC(y_deep)

0.94739711875107624

In [16]:
y_test=test[:,-1]
test_pred = model.predict(test[:,:8])
test_pred1 = deep.predict(test[:,:8])
print evalAUC(test_pred, y_test)
print evalAUC(test_pred1, y_test)

1.0
0.999169571218


In [47]:
rounded = np.round(test_pred)
rounded1 = np.round(test_pred1)
rounded.shape

(2789, 1)

In [155]:
def evaluate(y_pred, y_true):
    rounded = np.transpose(np.round(y_pred))
    tp = np.sum((rounded==1) & (y_true==1))
    fn = np.sum((rounded==0) & (y_true==1))
    tn = np.sum((rounded==0) & (y_true==0))
    fp = np.sum((rounded==1) & (y_true==0))
    sens = float(tp)/(tp+fn)
    spec = float(tn)/(tn+fp)
    acc = float(tp+tn)/(tp+fn+fp+tn)
    return (sens, spec, acc)
y_test=test[:,-1]
test_sens, test_spec, test_acc = evaluate(test_pred,y_test)
test_sens1, test_spec1, test_acc1 = evaluate(test_pred1,y_test)
print "Sensitivity: "+str(test_sens)+". Specificity: "+str(test_spec)+". Accuracy: "+str(test_acc)+"."
print "Deep Sensitivity: "+str(test_sens1)+". Deep Specificity: "+str(test_spec1)+". Deep Accuracy: "+str(test_acc1)+"."

Sensitivity: 1.0. Specificity: 1.0. Accuracy: 1.0.
Deep Sensitivity: 0.0. Deep Specificity: 1.0. Deep Accuracy: 0.988884904984.


In [156]:
val_true=val[:,-1]

val_sens, val_spec, val_acc = evaluate(y_pred,val_true)
val_sens1, val_spec1, val_acc1 = evaluate(y_deep,val[:,-1])
print "Sensitivity: "+str(val_sens)+". Specificity: "+str(val_spec)+". Accuracy: "+str(val_acc)+"."
print "Deep Sensitivity: "+str(val_sens1)+". Deep Specificity: "+str(val_spec1)+". Deep Accuracy: "+str(val_acc1)+"."

Sensitivity: 0.921052631579. Specificity: 0.99963649582. Accuracy: 0.998565794191.
Deep Sensitivity: 0.0. Deep Specificity: 1.0. Deep Accuracy: 0.986375044819.


In [158]:
(0.998565794191/0.986375044819-1

0.14113740140506925

In [159]:
model.save("TunedNN.hdf5")